In [1]:
import os,sys

import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.expand_frame_repr', False)

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from timeit import default_timer as timer
import re
import optuna
import matplotlib 
import matplotlib.pyplot as plt
from pathlib import Path

import os, sys, shutil

# 1) make sure you have a writable copy
shutil.copytree(
  '/kaggle/input/drfold2/DRfold2',
  '/kaggle/working/drfold',
  dirs_exist_ok=True
)

# 2) point Python at the cfg_97 folder under that copy
BASE = '/kaggle/working/drfold'
sys.path.insert(0, os.path.join(BASE, 'cfg_97'))
from EvoMSA2XYZ import MSA2XYZ
from RNALM2.Model import RNA2nd
from data import parse_seq, Get_base
from util import outpdb
# parse_pdb_to_xyz write_frame_coor_to_pdb

import numpy as np


# helper--
class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__

	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

def time_to_str(t, mode='min'):
	if mode=='min':
		t  = int(t)/60
		hr = t//60
		min = t%60
		return '%2d hr %02d min'%(hr,min) 
	elif mode=='sec':
		t   = int(t)
		min = t//60
		sec = t%60
		return '%2d min %02d sec'%(min,sec)

	else:
		raise NotImplementedError

def gpu_memory_use():
    if torch.cuda.is_available():
        device = torch.device(0)
        free, total = torch.cuda.mem_get_info(device)
        used= (total - free) / 1024 ** 3
        return int(round(used))
    else:
        return 0

def set_aspect_equal(ax):
	x_limits = ax.get_xlim()
	y_limits = ax.get_ylim()
	z_limits = ax.get_zlim()

	# Compute the mean of each axis
	x_middle = np.mean(x_limits)
	y_middle = np.mean(y_limits)
	z_middle = np.mean(z_limits)

	# Compute the max range across all axes
	max_range = max(x_limits[1] - x_limits[0],
					y_limits[1] - y_limits[0],
					z_limits[1] - z_limits[0]) / 2.0

	# Set the new limits to ensure equal scaling
	ax.set_xlim(x_middle - max_range, x_middle + max_range)
	ax.set_ylim(y_middle - max_range, y_middle + max_range)
	ax.set_zlim(z_middle - max_range, z_middle + max_range)


print('torch',torch.__version__)
print('torch.cuda',torch.version.cuda)

print('IMPORT OK!!!')

will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint


/kaggle/working/drfold/cfg_97/EvoMSA2XYZ.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  RNAlm.load_state_dict(torch.load(saved_model,map_location=torch.device('cpu')),

torch 2.5.1+cu121
torch.cuda 12.1
IMPORT OK!!!


In [2]:
# ─── TOP-OF-NOTEBOOK INTEGRATION CELL ───
CFG_FOLDER  = "/kaggle/input/drfold2/DRfold2/model_hub/cfg_97"
ROOT_FOLDER = "/kaggle/input/drfold2/DRfold2"
def write_frame_coor_to_pdb(coord, seq, savefile):
    """
    coord: np.ndarray of shape (L,3,3) from your model (P, sugar, N)
    seq:   string of length L
    savefile: path to write PDB with only C1' atom per residue
    """
    L = coord.shape[0]
    with open(savefile, 'w') as f:
        count = 1
        for i, res in enumerate(seq):
            x, y, z = coord[i, 1]   # channel=1 → sugar atom
            # PDB ATOM line building:
            # atom serial, atom name, residue name, chain A, residue seq, x,y,z, occup,temp, element
            f.write(
                f"ATOM  {count:5d}  C1' {res:>3s} A{ i+1:4d}"
                f"{x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           C\n"
            )
            count += 1
        f.write("TER\n")


# ── pure-Python FASTA I/O ──
def parse_fasta(path):
    recs, h, seqs = [], None, []
    with open(path) as f:
        for line in f:
            line = line.rstrip("\n")
            if line.startswith(">"):
                if h is not None:
                    recs.append((h, "".join(seqs)))
                h = line[1:].split()[0]
                seqs = []
            else:
                seqs.append(line)
        if h is not None:
            recs.append((h, "".join(seqs)))
    return recs

# ── load model & weights ──
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = dict(seq_dim=6, msa_dim=7, N_ensemble=1, N_cycle=8, m_dim=64, s_dim=64, z_dim=64)
model = MSA2XYZ(**config).to(device)

weights_path = os.path.join(CFG_FOLDER, "model_0")
model.load_state_dict(torch.load(weights_path, map_location=device), strict=False)
model.eval()

print("✅ DRfold2 + MSA integration ready. Device:", device)
print("   parse_seq → (L,6); parse_msa → (N,L,7); model loaded.")
# ─── end of integration cell ───



will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint


<ipython-input-2-02d836b58166>:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights_path, map_location=device), strict=False)


✅ DRfold2 + MSA integration ready. Device: cuda
   parse_seq → (L,6); parse_msa → (N,L,7); model loaded.


In [3]:
!pip install /kaggle/input/biopython/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


Processing /kaggle/input/biopython/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [4]:
#tuning
class SequenceStructureDataset(Dataset):
    def __init__(self, csv_file, data_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.data_dir = Path(data_dir)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        features = np.load(self.data_dir / f"{row['ID']}_features.npy")
        coords   = np.load(self.data_dir / f"{row['ID']}_coords.npy")
        sample = {'features': torch.from_numpy(features).float(),
                  'coords':   torch.from_numpy(coords).float()}
        return self.transform(sample) if self.transform else sample


DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'

train_dataset = SequenceStructureDataset(
    csv_file=f"{DATA_KAGGLE_DIR}/train_labels.csv",
    data_dir=f"{DATA_KAGGLE_DIR}/train_data"
)
val_dataset = SequenceStructureDataset(
    csv_file=f"{DATA_KAGGLE_DIR}/validation_labels.csv",
    data_dir=f"{DATA_KAGGLE_DIR}/validation_data"
)

In [ ]:
# from datetime import datetime
# import pytz
# print('LOGGING TIME OF START:',  datetime.strftime(datetime.now(pytz.timezone('Asia/Singapore')), "%Y-%m-%d %H:%M:%S"))


# try:
#     import Bio
# except:
#     #for drfold2 --------
#     #!pip install biopython
#     !pip install /kaggle/input/biopython/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install /kaggle/input/biopython/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install biopython
# print('PIP INSTALL OK !!!!')

In [5]:
# MODE = 'local' #'local' # submit
MODE = 'local'
MSA_DIR = '/kaggle/input/stanford-rna-3d-folding/MSA_v2'

DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'
if MODE == 'local':
    valid_df = pd.read_csv("/kaggle/input/validation-sequences-clean-csv/validation_sequences_clean.csv")
    label_df = pd.read_csv("/kaggle/input/validation-labels-clean-csv/validation_labels_clean.csv")
    label_df['target_id'] = label_df['ID'].apply(lambda x: '_'.join(x.split('_')[:-1]))

if MODE == 'submit':
	valid_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/test_sequences.csv')

print('len(valid_df)',len(valid_df))
print(valid_df.iloc[0])
print('')


# cfg = dotdict(
#     num_conf = 5,
#     max_length=480,
# )
NUM_CONF=5
MAX_LENGTH=480
DEVICE='cuda' #'cpu'

print('MODE:', MODE)
print('SETTING OK!!!')

len(valid_df) 94
target_id                                                     9L5R_2
sequence           AGCUCUCUUUGCCUUUUGGCUUAGAUCAAGUGUAGUAUCUGUUCUU...
temporal_cutoff                                           2025-03-12
description        Cryo-EM structure of the thermophile spliceoso...
all_sequences      >9L5R_1|Chain A[auth 2]|U2 snRNA|Chaetomium th...
Name: 0, dtype: object

MODE: local
SETTING OK!!!


In [6]:
import os
os.makedirs('/kaggle/working/drfold', exist_ok=True)
!cp -r /kaggle/input/drfold2/DRfold2/* /kaggle/working/drfold/

In [ ]:
# import sys, os
# CFG97 = os.path.join('/kaggle/input/drfold/DRfold2/DRfold2', 'cfg_97')
# assert os.path.isdir(CFG97), f"{CFG97} not found!"
# sys.path.insert(0, CFG97)

# # now this should succeed:
# from EvoMSA2XYZ import MSA2XYZ
# from RNALM2.Model import RNA2nd
# from data         import parse_seq, Get_base, BASE_COOR
# from data         import write_frame_coor_to_pdb, parse_pdb_to_xyz

# print("imported!", MSA2XYZ, RNA2nd)

In [7]:
import os
import numpy as np

# 1) Point at the cfg_97 folder (adjust this to your actual path)
CFG97 = '/kaggle/input/drfold2/DRfold2/cfg_97'

# 2) Load the base.npy file once into a Python variable
BASE_COOR = np.load(os.path.join(CFG97, 'base.npy'))

# 3) Now import your parsing and model code
import sys
sys.path.insert(0, CFG97)
from data import parse_seq, Get_base
# (no BASE_COOR to import from data.py)

# 4) When you need the 3×3 base coordinates for a sequence, call:
sequence = "ACGUACGUA"
base_coords = Get_base(sequence, BASE_COOR)
# base_coords.shape == (len(sequence), 3, 3)

In [13]:
def parse_pdb_to_xyz(pdb_file, atom_name=" P  "):
    coords, resid, resname = [], [], []
    with open(pdb_file) as f:
        for line in f:
            if line.startswith("ATOM") and line[12:16].strip() == atom_name.strip():
                x = float(line[30:38]); y = float(line[38:46]); z = float(line[46:54])
                coords.append((x,y,z))
                resid.append(int(line[22:26]))
                resname.append(line[17:20].strip())
    return np.array(coords, dtype=np.float32), resname, resid


def parse_msa(path: str, max_seqs: int = 100) -> np.ndarray:
    """
    Reads a FASTA/A3M MSA, caps at max_seqs rows, one-hots each → (L,6),
    and stacks into (N,L,6) - no target marker channel for now.
    """
    recs = parse_fasta(path)[:max_seqs]
    if not recs:
        raise ValueError(f"no sequences in MSA at {path}")
    N = len(recs)
    L = len(recs[0][1])
    msa = np.zeros((N, L, 6), dtype=np.float32)  # Changed from 7 to 6
    
    for i, (_, seq) in enumerate(recs):
        # Always use our one-hot function for MSA
        sq = parse_seq_onehot(seq)
        if i < 5:  # Only print first 5 rows to reduce spam
            print(f"   Row {i}: seq length {len(seq)}, parse_seq_onehot shape {sq.shape}, expected L={L}")
        
        if sq.shape[0] != L:
            raise ValueError(f"row {i} length {sq.shape[0]} != expected {L}")
        if sq.shape[1] != 6:
            raise ValueError(f"row {i} features {sq.shape[1]} != expected 6")
        
        msa[i,:,:6] = sq
        # Remove the target marker for now
    return msa

# ── one-hot encoder for a single RNA sequence ──
def parse_seq_onehot(seq: str) -> np.ndarray:
    """
    Returns (L,6) float32 array:
      ch0–3 = one-hot A/G/C/U,
      ch4   = everything else (gaps/unknown),
      ch5   = mask = 1.0 everywhere.
    """
    L = len(seq)
    feat = np.zeros((L,6), dtype=np.float32)
    cmap = {'A':0, 'G':1, 'C':2, 'U':3, 'T':3}
    for i, r in enumerate(seq):
        feat[i, cmap.get(r,4)] = 1.0
    feat[:,5] = 1.0
    return feat

In [14]:

###########################################################3
KAGGLE_TRUTH_PDB_DIR ='/kaggle/working/drfold/kaggle-casp15-truth'
USALIGN = '/kaggle/working/USalign' 
os.system('cp /kaggle/input/usalign/USalign /kaggle/working/')
os.system('sudo chmod u+x /kaggle/working/USalign')

# evaluate helper
def get_truth_df(target_id, label_df):
    truth_df = label_df[label_df['target_id'] == target_id]
    truth_df = truth_df.reset_index(drop=True)
    return truth_df

def parse_usalign_for_tm_score(output):
    # Extract TM-score based on length of reference structure (second)
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)[1]
    if not tm_score_match:
        raise ValueError('No TM score found')
    return float(tm_score_match)

def parse_usalign_for_transform(output):
    # Locate the rotation matrix section
    matrix_lines = []
    found_matrix = False

    for line in output.splitlines():
        if "The rotation matrix to rotate Structure_1 to Structure_2" in line:
            found_matrix = True
        elif found_matrix and re.match(r'^\d+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+$', line):
            matrix_lines.append(line)
        elif found_matrix and not line.strip():
            break  # Stop parsing if an empty line is encountered after the matrix

    # Parse the rotation matrix values
    rotation_matrix = []
    for line in matrix_lines:
        parts = line.split()
        row_values = list(map(float, parts[1:]))  # Skip the first column (index)
        rotation_matrix.append(row_values)
    return np.array(rotation_matrix)



# data helper
def make_data(seq, target_id, msa_dir):
    base = Get_base(seq, BASE_COOR)
    seq_idx = np.arange(len(seq)) + 1
    
    # Try to load MSA, fallback to single sequence if not found
    msa_file = os.path.join(msa_dir, f"{target_id}.MSA.fasta")
    
    if os.path.exists(msa_file):
        print(f"✅ Loading MSA from {msa_file}")
        print(f"   Target sequence length: {len(seq)}")
        try:
            msa_data = parse_msa(msa_file, max_seqs=100)  # Shape: (N,L,7)
            print(f"   MSA loaded: {msa_data.shape[0]} sequences, {msa_data.shape[1]} positions")
            
            # Verify MSA matches sequence length
            if msa_data.shape[1] != len(seq):
                print(f"⚠️ MSA length {msa_data.shape[1]} != sequence length {len(seq)}, using single sequence")
                msa_data = create_single_sequence_msa(seq)
            else:
                # Verify target sequence matches first row of MSA
                target_from_msa = msa_to_sequence(msa_data[0, :, :6])
                if not sequences_match(target_from_msa, seq):
                    print(f"⚠️ Target sequence mismatch with MSA, using single sequence")
                    msa_data = create_single_sequence_msa(seq)
        except Exception as e:
            print(f"⚠️ Error loading MSA: {e}, using single sequence")
            msa_data = create_single_sequence_msa(seq)
    else:
        print(f"⚠️ MSA file not found: {msa_file}, using single sequence")
        msa_data = create_single_sequence_msa(seq)
    
    msa = torch.from_numpy(msa_data).float()  # Shape: (N,L,7)
    base_x = torch.from_numpy(base).float()
    seq_idx = torch.from_numpy(seq_idx).long()
    
    return msa, base_x, seq_idx

def create_single_sequence_msa(seq):
    """Create a fake MSA with just the target sequence duplicated"""
    # Always use our one-hot function
    aa_type = parse_seq_onehot(seq)  
    print(f"   aa_type shape: {aa_type.shape}")
    
    # Create MSA with 2 identical sequences, 6 channels only
    msa_data = np.zeros((2, len(seq), 6), dtype=np.float32)  # Changed from 7 to 6
    msa_data[0, :, :6] = aa_type  # First sequence
    msa_data[1, :, :6] = aa_type  # Duplicate
    # Remove target marker for now
    return msa_data

def msa_to_sequence(msa_row):
    """Convert one-hot MSA row back to sequence string"""
    seq = ""
    for i in range(msa_row.shape[0]):
        if msa_row[i, 0] == 1.0: seq += "A"
        elif msa_row[i, 1] == 1.0: seq += "G"
        elif msa_row[i, 2] == 1.0: seq += "C"
        elif msa_row[i, 3] == 1.0: seq += "U"
        else: seq += "N"  # Unknown/gap
    return seq

def sequences_match(seq1, seq2, gap_chars="-N"):
    """Check if sequences match, ignoring gaps"""
    s1 = ''.join(c for c in seq1.upper() if c not in gap_chars)
    s2 = ''.join(c for c in seq2.upper() if c not in gap_chars)
    return s1 == s2
    
def make_dummy_solution():
    solution=dotdict()
    for i, row in valid_df.iterrows():
        target_id = row.target_id
        sequence = row.sequence
        solution[target_id]=dotdict(
            target_id=target_id,
            sequence=sequence,
            coord=[],
        )
    return solution

def solution_to_submit_df(solution):
    submit_df = []
    for k,s in solution.items():
        df = coord_to_df(s.sequence, s.coord, s.target_id)
        submit_df.append(df)
    
    submit_df = pd.concat(submit_df)
    return submit_df
 

def coord_to_df(sequence, coord, target_id):
    L = len(sequence)
    df = pd.DataFrame()
    df['ID'] = [f'{target_id}_{i + 1}' for i in range(L)]
    df['resname'] = [s for s in sequence]
    df['resid'] = [i + 1 for i in range(L)]

    num_coord = len(coord)
    for j in range(num_coord):
        df[f'x_{j+1}'] = coord[j][:, 0]
        df[f'y_{j+1}'] = coord[j][:, 1]
        df[f'z_{j+1}'] = coord[j][:, 2]
    return df

In [10]:
# Add this diagnostic code to check MSA availability
import os

msa_directory = "/kaggle/input/stanford-rna-3d-folding/MSA_v2"
test_data = pd.read_csv("/kaggle/input/validation-sequences-clean-csv/validation_sequences_clean.csv")

print(f"MSA directory exists: {os.path.exists(msa_directory)}")

if os.path.exists(msa_directory):
    msa_files = os.listdir(msa_directory)
    print(f"Number of MSA files found: {len(msa_files)}")
    print(f"First 10 MSA files: {msa_files[:10]}")
    
    # Check what target IDs we're looking for
    target_ids = test_data['target_id'].tolist()
    print(f"First 10 target IDs: {target_ids}")
    
    # Check if any MSA files match our target IDs
    matches = []
    for target_id in target_ids:
        expected_file = f"{target_id}.MSA.fasta"
        if expected_file in msa_files:
            matches.append(expected_file)
    
    print(f"Matching MSA files found: {len(matches)}")

else:
    print("MSA directory does not exist!")
    print("Available directories in /kaggle/input/stanford-rna-3d-folding/:")
    if os.path.exists("/kaggle/input/stanford-rna-3d-folding/"):
        print(os.listdir("/kaggle/input/stanford-rna-3d-folding/"))
    else:
        print("Stanford RNA 3D folding dataset not available!")

MSA directory exists: True
Number of MSA files found: 2534
First 10 MSA files: ['3JCS_6.MSA.fasta', '7MSF_R.MSA.fasta', '2OOM_B.MSA.fasta', '1ZDI_S.MSA.fasta', '5FJ1_H.MSA.fasta', '6UF1_C.MSA.fasta', '4V8A_AB.MSA.fasta', '6JDG_G.MSA.fasta', '5NCO_1.MSA.fasta', '5DI4_A.MSA.fasta']
First 10 target IDs: ['9L5R_2', '9GFT_AU', '9L0R_K', '9GFT_A3', '9B2K_B', '9B0S_Et', '9J3T_B', '9LCR_B', '8KEB_A', '9L5S_5', '8VXZ_C', '9J6Y_E', '8QHU_5', '9GHF_Z', '9KPO_B', '9N2B_5', '9N2C_Pt', '9B1Y_4', '9G06_a', '9DE8_A', '9B83_C', '8ZMH_A', '9E2Y_F', '9DE7_A', '8Y9L_B', '9FIB_Y', '9J3R_B', '9DPB_C', '8XTP_A', '8ZTV_Y', '8Y9M_B', '8ZQ9_A', '8XTP_B', '9B89_C', '8SYK_C', '9FN3_B', '8QHU_3', '9DRS_C', '8XTR_A', '9LMF_F', '9DE6_B', '8SYK_B', '9DE6_A', '8R7N_A', '8K85_A', '9FCV_B', '9DPA_C', '9DE5_C', '8VZ6_S', '8YIG_C', '9B84_F', '9C8K_2', '9B0Q_AP', '9E2Z_F', '8Z8Q_B', '9E2W_F', '8KHH_A', '8Z8U_B', '8ZTU_Y', '9GCL_A', '8RRI_Ax', '9L5S_6', '9GCM_A', '8Z9K_B', '9MTY_C', '8QHU_7', '9GBW_R', '8T5O_A', '9DPL_C', '

In [15]:
################### start here !!! #######################################################3
out_dir = '/kaggle/working/model-output'
os.makedirs(out_dir, exist_ok=True)
solution = make_dummy_solution()


#load model (these are moified versions, not the same from their github repo)
rnalm = RNA2nd(dict(
    s_in_dim=5,
    z_in_dim=2,
    s_dim= 512,
    z_dim= 128,
    N_elayers=18,
))
rnalm_file = '/kaggle/working/drfold/model_hub/RCLM/epoch_67000'
print(rnalm_file)
print(
    rnalm.load_state_dict(torch.load(rnalm_file, map_location='cpu', weights_only=True), strict=False)
    #Unexpected key(s) in state_dict: "ss_head.linear.weight", "ss_head.linear.bias".
)
rnalm = rnalm.to(DEVICE)
rnalm = rnalm.eval()
total_time_taken = 0
max_gpu_mem_used = 0

will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
/kaggle/working/drfold/model_hub/RCLM/epoch_67000
_IncompatibleKeys(missing_keys=[], unexpected_keys=['ss_head.linear.weight', 'ss_head.linear.bias'])


In [ ]:
cfg = dict(
    seq_dim=6,
    msa_dim=7,
    N_ensemble=1,   # how many ensemble members
    N_cycle=8,      # how many recycling cycles
    m_dim=64,
    s_dim=64,
    z_dim=64,
)
for c in range(NUM_CONF): 
    msa2xyz = MSA2XYZ(**cfg)
    msa2xyz_file = [
        f'/kaggle/working/drfold/model_hub/cfg_97/model_{k}' for k in [0,1,2,8,9]
    ][c]
    print(msa2xyz_file)
    print(
        msa2xyz.load_state_dict(torch.load(msa2xyz_file, map_location='cpu', weights_only=True), strict=True)
    )
    msa2xyz.msaxyzone.premsa.rnalm = rnalm
    msa2xyz = msa2xyz.to(DEVICE)
    msa2xyz = msa2xyz.eval()
 
    for i,row in valid_df.iterrows():
        start_timer = timer()
        
        target_id = row.target_id
        sequence = row.sequence
        seq = row.sequence    
        
        L = len(sequence)
        if L>MAX_LENGTH:
            i0 = np.random.choice(L-MAX_LENGTH+1)
            i1 = i0 + MAX_LENGTH
        else:
            i0 = 0
            i1 = L
        
        seq = sequence[i0:i1]
        print(c,i,target_id, L, seq[:75]+'...')
        
        msa, base_x, seq_idx = make_data(seq, target_id, MSA_DIR)
        msa, base_x, seq_idx = msa.to(DEVICE), base_x.to(DEVICE), seq_idx.to(DEVICE)
        
        # MSA usage verification
        print(f"   MSA tensor shape: {msa.shape}")
        if msa.shape[0] > 2:
            print(f"   ✅ Using real MSA with {msa.shape[0]} sequences")
        else:
            print(f"   ⚠️ Using single sequence (duplicated)")
            
        # Extract sequence features (first 6 channels) for the model
        # The model expects seq to be (L, 6), not (L, 7)
        seq_features = msa[0, :, :6]  # Take first row, first 6 channels
        print(f"   Seq features shape: {seq_features.shape}")
            
        secondary = None #secondary structure
    
        with torch.no_grad(): 
            out = msa2xyz.pred(msa, seq_idx, secondary, base_x, np.array(list(seq)))

        # key = list(out.keys()) # plddt(L,L), coor(L,3,3), dist_p(L,L,38), dist_c, dist_n,
        # for k in key:
        #     print(k, type(out[k]), out[k].shape)
 
        
        if L!=len(seq):
             out['coor'] = np.pad(out['coor'] ,((i0, L - i1), (0, 0), (0, 0)), 'constant', constant_values=0)


        print('out:',  out['coor'].shape)
        
        # Log MSA usage statistics
        msa_depth = msa.shape[0]
        if msa_depth > 2:
            print(f"   📊 Processed with MSA depth: {msa_depth}")
        else:
            print(f"   📊 Processed with single sequence (no MSA)")
            
        time_taken = timer()-start_timer
        time_taken = timer()-start_timer
        total_time_taken += time_taken
        print('time_taken:', time_to_str(time_taken, mode='sec')) 
        
        gpu_mem_used = gpu_memory_use()
        max_gpu_mem_used = max(max_gpu_mem_used,gpu_mem_used)
        print('gpu_mem_used:', gpu_mem_used, 'GB')

        torch.cuda.empty_cache() 
        sugar_xyz = out['coor'][:, 1, :]   # shape (L,3)
        solution[target_id].coord.append(sugar_xyz)
    print('')
    
#-----end of conformation generation ----
print('MAX_LENGTH', MAX_LENGTH)
print('### total_time_taken:', time_to_str(total_time_taken, mode='min'))
print('### max_gpu_mem_used:', max_gpu_mem_used, 'GB')

# MSA usage summary
msa_used_count = 0
total_targets = len(valid_df)
for i, row in valid_df.iterrows():
    target_id = row.target_id
    msa_file = os.path.join(MSA_DIR, f"{target_id}.MSA.fasta")
    if os.path.exists(msa_file):
        msa_used_count += 1

print(f'### MSA usage: {msa_used_count}/{total_targets} targets had MSA files')
print(f'### MSA coverage: {msa_used_count/total_targets*100:.1f}%')
print('')

submit_df = solution_to_submit_df(solution)
submit_df.to_csv(f'submission.csv', index=False)
print(submit_df)
print('SUBMIT OK!!!!!!')
print('')


will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
will do checkpoint
/kaggle/working/drfold/model_hub/cfg_97/model_0
<All keys matched successfully>
0 0 9L5R_2 193 AGCUCUCUUUGCCUUUUGGCUUAGAUCAAGUGUAGUAUCUGUUCUUUUCAGUUUAAUCUCUGAAACUGCUCUACG...
✅ Loading MSA from /kaggle/input/stanford-rna-3d-folding/MSA_v2/9L5R_2.MSA.fasta
   Target sequence length: 193
   Row 0: seq length 193, parse_seq_onehot shape (193, 6), expected L=193
   Row 1: seq length 193, parse_seq_onehot shape (193, 6), expected L=193
   Row 2: seq length 193, parse_seq_onehot shape (193, 6), expected L=193
   Row 3: seq length 193, parse_seq_onehot shape (193, 6), expected L=193
   Row 4: seq length 193, parse_seq_onehot shape (193, 6), expected L=193
   MSA loaded: 100 sequences, 193 positi

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


out: (193, 3, 3)
   📊 Processed with MSA depth: 100
time_taken:  0 min 12 sec
gpu_mem_used: 6 GB
0 1 9GFT_AU 76 GGGGCUAUAGCUCAGCUGGGAGAGCGCUUGCAUGGCAUGCAAGAGGUCAGCGGUUCGAUCCCGCUUAGCUCCACC...
✅ Loading MSA from /kaggle/input/stanford-rna-3d-folding/MSA_v2/9GFT_AU.MSA.fasta
   Target sequence length: 76
   Row 0: seq length 76, parse_seq_onehot shape (76, 6), expected L=76
   Row 1: seq length 76, parse_seq_onehot shape (76, 6), expected L=76
   Row 2: seq length 76, parse_seq_onehot shape (76, 6), expected L=76
   Row 3: seq length 76, parse_seq_onehot shape (76, 6), expected L=76
   Row 4: seq length 76, parse_seq_onehot shape (76, 6), expected L=76
   MSA loaded: 100 sequences, 76 positions
   MSA tensor shape: torch.Size([100, 76, 6])
   ✅ Using real MSA with 100 sequences
   Seq features shape: torch.Size([76, 6])
out: (76, 3, 3)
   📊 Processed with MSA depth: 100
time_taken:  0 min 02 sec
gpu_mem_used: 3 GB
0 2 9L0R_K 700 CUGACGAAGUUCAAGGGUGGUACCAAGAGCGGUCAGCAUGUUGUGCAACGCUGGGAGGAU

In [ ]:
submit_df = solution_to_submit_df(solution)
submit_df.to_csv(f'submission.csv', index=False)
print(submit_df)
print('SUBMIT OK!!!!!!')
print('')